### `CosineSimilarity` code

In [34]:
import numpy as np
import torch # needed for tests
from tqdm import tqdm # needed for tests

In [35]:
class CosineSimilarity:
    ''' Computes CosineSimilarity Loss given the input1 and input2 '''
    
    ''' * The class implementation will be along the lines of torch.nn.CosineSimilarity in order to 
          enable comparison of this NumPy only implementation and seamless testing
        * Can expect extensive refactoring of the existing code in the days to come
        * As part of refactoring, some code will be de-modularized
        * Old code will be retained at the end of the notebook for reference
    '''
    '''
        TODO:
        * Replace `torch.round()` with `np.allclose()` for tests
        * Optimizing code
    '''
    
    def __init__(
        self,
        dim = 1,
        eps = 1e-08,
        verbose = False
        ):
        super(CosineSimilarity, self).__init__()
        
        ''' mandatory parameters '''
        # None
        
        ''' optional parameters '''
        self.dim = dim
        self.eps = eps
        
        ''' optional parameters (dummy, yet to be implemented)'''
        # None
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('dim: {}, eps: {}'.format(self.dim, self.eps))
        self.verboseprint('\n')
    
    def forward(self, _input1, _input2):
        ''' forward pass to compute CosineSimilarity'''
        
        ''' error checking '''
        if not (isinstance(_input1, int) or isinstance(_input1, float) or isinstance(_input1, np.ndarray)):
            raise Exception('invalid input: `input1` should either be an int, a float, or a NumPy ndarray')
        
        if not (isinstance(_input2, int) or isinstance(_input2, float) or isinstance(_input2, np.ndarray)):
            raise Exception('invalid input: `input2` should either be an int, a float, or a NumPy ndarray')
            
        if (isinstance(_input1, np.ndarray) and not isinstance(_input2, np.ndarray))  or (not isinstance(_input1, np.ndarray) and isinstance(_input2, np.ndarray)):
            raise Exception('invalid input: `input1` and `input2` should both be a NumPy ndarray, or can be a mix of `int` and `float`')
        
        if (len(_input1.shape) <= self.dim):
            self.dim = len(_input1.shape) - 1
        
        if (len(_input2.shape) <= self.dim):
            self.dim = len(_input2.shape) - 1
        
        ''' compute CosineSimilarity '''
        dot_product = np.sum(_input1 * _input2, axis = self.dim)
        norm1 = np.sqrt(np.sum(_input1 ** 2, axis = self.dim))
        norm2 = np.sqrt(np.sum(_input2 ** 2, axis = self.dim))
        output = dot_product / (norm1 * norm2)
        self.verboseprint("*** CosineSimilarity output ***")
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random input, target)

In [36]:
num_dim = np.random.randint(6) + 1 # number of input dimensions
shape = tuple(np.random.randint(5) + 1 for _ in range(num_dim)) # shape of input
_input1 = np.random.rand(*shape) # generate an input based on the dimensions and shape
_input2 = np.random.rand(*shape) # generate another input based on the dimensions and shape

In [37]:
# compute CosineSimilarity using the random input and target
cosinesimilarity = CosineSimilarity() # call an instance of the class
_output = cosinesimilarity.forward(_input1, _input2) # compute CosineSimilarity
print("*** CosineSimilarity output ***")
print(_output)

*** CosineSimilarity output ***
[[[[0.6787122  0.96391397 0.59424573 0.94292907]]

  [[0.89879905 0.95325673 0.44959406 0.81565049]]]]


In [38]:
# get PyTorch output with the same random inputs as above

x = torch.DoubleTensor(_input1)
y = torch.DoubleTensor(_input2)
loss = torch.nn.CosineSimilarity()
output = loss(x, y)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[0.6787, 0.9639, 0.5942, 0.9429]],

         [[0.8988, 0.9533, 0.4496, 0.8157]]]], dtype=torch.float64)


In [39]:
# compare outputs of CosineSimilarity and PyTorch
if not isinstance(_output, np.ndarray): # if a singleton, convert PyTorch tensor to NumPy float, round, and compare
    output = output.numpy()
    print(np.equal(np.round(_output), np.round(output)))  
else:
    print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests (random input, target)

In [40]:
def run_tests(num_tests):
    ''' sweep different input parameters and test by comparing outputs of ReLU and PyTorch '''
    
    num_passed = 0
    print('Number of tests: {}\n\n'.format(num_tests))
    
    for i in tqdm(range(num_tests)):
        num_dim = np.random.randint(6) + 1 # number of input dimensions
        shape = tuple(np.random.randint(5) + 1 for _ in range(num_dim)) # shape of input
        _input1 = np.random.rand(*shape) # generate an input based on the dimensions and shape
        _input2 = np.random.rand(*shape) # generate another input based on the dimensions and shape
        _dim = np.random.randint(num_dim) # dimension along which CosineSimilarity is to be computed
        _eps = np.random.uniform(low = 1e-10, high = 1e-6)
        print('Test: {}\nInput shape: {}, Dimension: {}, eps: {}'.format(i, shape, _dim, _eps))
        
        try:
            # compute CosineSimilarity using the random input and target
            cosinesimilarity = CosineSimilarity(dim = _dim, eps = _eps) # call an instance of the class
            _output = cosinesimilarity.forward(_input1, _input2) # compute CosineSimilarity

            # get PyTorch output with the same random inputs as above
            x = torch.DoubleTensor(_input1)
            y = torch.DoubleTensor(_input2)
            loss = torch.nn.CosineSimilarity(dim = _dim, eps = _eps)
            output = loss(x, y)

            
        except Exception as e:
            print(e)
            print('Result: False\n\n') # treating exception as a failed test
            continue

        # compare outputs of CosineSimilarity and PyTorch
        if not isinstance(_output, np.ndarray): # if a singleton, convert PyTorch tensor to NumPy float, round, and compare
            output = output.numpy()
            result = np.equal(np.round(_output), np.round(output)) 
        else:
            result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
        print('Result: {}\n\n'.format(result))
        if result:
            num_passed += 1

    print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))


In [41]:
num_tests = 10000
run_tests(num_tests)

Number of tests: 10000




  0%|                                                 | 0/10000 [00:00<?, ?it/s]

Test: 0
Input shape: (5, 5, 4, 1, 4, 1), Dimension: 5, eps: 4.835058993983436e-07
Result: True


Test: 1
Input shape: (5, 5, 3, 2, 1), Dimension: 2, eps: 3.036750063609143e-07
Result: True


Test: 2
Input shape: (3, 2), Dimension: 1, eps: 6.256840800186509e-07
Result: True


Test: 3
Input shape: (3, 1, 4), Dimension: 0, eps: 1.494591384183607e-07
Result: True


Test: 4
Input shape: (2, 4, 3), Dimension: 0, eps: 6.801387892518647e-07
Result: True


Test: 5
Input shape: (4, 1, 5, 5, 5), Dimension: 0, eps: 8.267505390394222e-07
Result: True


Test: 6
Input shape: (5, 4, 3, 1), Dimension: 3, eps: 8.363114349017134e-07
Result: True


Test: 7
Input shape: (2, 3, 4, 5), Dimension: 3, eps: 6.639564635061859e-07
Result: True


Test: 8
Input shape: (2, 1, 3, 3, 5), Dimension: 0, eps: 7.712162109198031e-07
Result: True


Test: 9
Input shape: (5, 2, 5), Dimension: 1, eps: 4.542110536813827e-07
Result: True


Test: 10
Input shape: (1, 3, 3), Dimension: 2, eps: 2.2480211444973028e-07
Result: True




  1%|▍                                    | 103/10000 [00:00<00:09, 1029.65it/s]

Result: True


Test: 67
Input shape: (3, 3, 1), Dimension: 1, eps: 5.31188238629891e-07
Result: True


Test: 68
Input shape: (2, 2, 5, 2, 1), Dimension: 3, eps: 1.914663775115881e-07
Result: True


Test: 69
Input shape: (3,), Dimension: 0, eps: 3.8104205190164905e-07
Result: True


Test: 70
Input shape: (4, 3, 4, 1), Dimension: 2, eps: 6.180522488747635e-09
Result: True


Test: 71
Input shape: (1, 3, 3, 3), Dimension: 1, eps: 7.158293542859761e-07
Result: True


Test: 72
Input shape: (5, 1, 3, 5), Dimension: 2, eps: 6.000012711724142e-08
Result: True


Test: 73
Input shape: (5, 2, 4, 2), Dimension: 0, eps: 7.969704284377314e-07
Result: True


Test: 74
Input shape: (1, 3, 1, 5, 5), Dimension: 2, eps: 2.90482364449714e-07
Result: True


Test: 75
Input shape: (3, 2, 5), Dimension: 0, eps: 3.768282626472721e-08
Result: True


Test: 76
Input shape: (2, 5, 3, 3, 2, 5), Dimension: 0, eps: 4.073398999757541e-08
Result: True


Test: 77
Input shape: (3, 1, 4, 5), Dimension: 0, eps: 8.01325025464

...

Result: True


Test: 9707
Input shape: (4, 5, 3), Dimension: 2, eps: 7.204276090276322e-07
Result: True


Test: 9708
Input shape: (2,), Dimension: 0, eps: 7.207419235626204e-07
Result: True


Test: 9709
Input shape: (2, 1, 5), Dimension: 2, eps: 5.066520145375458e-07
Result: True


Test: 9710
Input shape: (4, 1), Dimension: 0, eps: 3.6233049613730885e-07
Result: True


Test: 9711
Input shape: (4, 1, 2), Dimension: 0, eps: 5.015488168705289e-07
Result: True


Test: 9712
Input shape: (5, 5, 3, 4), Dimension: 3, eps: 5.812192158321503e-07
Result: True


Test: 9713
Input shape: (4, 2, 3, 2, 2, 3), Dimension: 0, eps: 7.012272509976949e-07
Result: True


Test: 9714
Input shape: (3, 1, 2), Dimension: 1, eps: 9.968042875972318e-08
Result: True


Test: 9715
Input shape: (1, 5, 4, 2), Dimension: 2, eps: 5.780832868275835e-07
Result: True


Test: 9716
Input shape: (4,), Dimension: 0, eps: 6.616967082160207e-07
Result: True


Test: 9717
Input shape: (2,), Dimension: 0, eps: 2.3071848749549917e-07


 99%|███████████████████████████████████▊| 9943/10000 [00:06<00:00, 1656.84it/s]

Result: True


Test: 9926
Input shape: (2,), Dimension: 0, eps: 8.687152228150257e-07
Result: True


Test: 9927
Input shape: (2, 4, 5, 5, 2, 2), Dimension: 1, eps: 6.204252106630593e-07
Result: True


Test: 9928
Input shape: (4, 4), Dimension: 1, eps: 5.575006858827378e-07
Result: True


Test: 9929
Input shape: (5, 2, 2, 1), Dimension: 3, eps: 3.13154114167535e-08
Result: True


Test: 9930
Input shape: (1, 2, 1, 5), Dimension: 2, eps: 9.056240252046241e-08
Result: True


Test: 9931
Input shape: (4, 3, 4), Dimension: 2, eps: 1.2945991974182645e-07
Result: True


Test: 9932
Input shape: (2, 3), Dimension: 0, eps: 2.680258497929048e-07
Result: True


Test: 9933
Input shape: (4, 4, 3, 3, 2), Dimension: 0, eps: 6.1074205405491e-07
Result: True


Test: 9934
Input shape: (5, 2, 1, 1, 3), Dimension: 0, eps: 2.2084796358569093e-07
Result: True


Test: 9935
Input shape: (3,), Dimension: 0, eps: 2.1034392959820474e-07
Result: True


Test: 9936
Input shape: (2, 4, 5), Dimension: 2, eps: 8.07808181

100%|███████████████████████████████████| 10000/10000 [00:07<00:00, 1414.39it/s]

Result: True


Test: 9983
Input shape: (5, 5, 5, 3, 1), Dimension: 3, eps: 3.959152521506411e-07
Result: True


Test: 9984
Input shape: (3, 5), Dimension: 1, eps: 5.066434033391753e-07
Result: True


Test: 9985
Input shape: (1, 3, 4, 3), Dimension: 3, eps: 6.697856265050724e-07
Result: True


Test: 9986
Input shape: (4, 1, 3, 3, 5, 2), Dimension: 0, eps: 8.156684647846293e-08
Result: True


Test: 9987
Input shape: (4, 2), Dimension: 0, eps: 3.981164163266116e-07
Result: True


Test: 9988
Input shape: (1, 2, 2), Dimension: 2, eps: 2.8031605991993714e-07
Result: True


Test: 9989
Input shape: (1, 4), Dimension: 1, eps: 2.0354796121942085e-07
Result: True


Test: 9990
Input shape: (4, 2, 5), Dimension: 1, eps: 3.7911647084213723e-07
Result: True


Test: 9991
Input shape: (4, 1, 5, 2, 2, 5), Dimension: 4, eps: 8.141873933934231e-08
Result: True


Test: 9992
Input shape: (4, 4, 3, 4), Dimension: 1, eps: 6.445572158198545e-07
Result: True


Test: 9993
Input shape: (2, 1, 3), Dimension: 2, ep